.. I am thinking to not use building_id but instead:
- mean meter reading by meter type
- std of meter reading by meter type
- mean meter reading by meter type by week of year
- std meter reading by meter type by week of year

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from tsm.data_selector import data_subset_by_dict
import matplotlib.pyplot as plt
import numpy as np

In [3]:
train_data = pd.read_pickle('kaggle/input/ashrae-energy-prediction/train.pkl')

## Level 4A)
* 1 site - 124 buildings
* 1 meter reading - meter 0
* Building ID

In [7]:
level_4a = {'site_id': 14, 'meter': 0}
level_4a_cols = ['building_id', 'meter_reading', 'dt_m', 'dt_w', 'dt_d', 'dt_hour', 
                 'dt_day_week', 'dt_day_month', 'dt_week_month']

In [8]:
data_4a = data_subset_by_dict(train_data, level_4a, level_4a_cols)

In [9]:
data_4a.head()

,building_id,meter_reading,dt_m,dt_w,dt_d,dt_hour,dt_day_week,dt_day_month,dt_week_month
1778,1223,79.845299,1,53,1,0,4,1,1
1781,1224,12.000000,1,53,1,0,4,1,1
1784,1225,185.647003,1,53,1,0,4,1,1
1787,1226,42.340900,1,53,1,0,4,1,1
1790,1227,52.820000,1,53,1,0,4,1,1


In [13]:
import lightgbm as lgb

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [14]:
from tsm.eval_metrics import root_mean_squared_log_error

In [16]:
SEED = 7

In [20]:
lgb_reg_params = {'objective':'regression',  'boosting_type':'gbdt', 'metric':'rmse',
                  'n_jobs':-1, 'learning_rate':0.07, 'num_leaves': 2**8, 'max_depth':-1,
                  'tree_learner':'serial', 'colsample_bytree': 0.7, 'subsample_freq':1,
                  'subsample':0.5, 'max_bin':255, 'verbose':1, 'seed': SEED, 'early_stopping_rounds': 40} 

In [11]:
# Cross Validation
from sklearn.model_selection import KFold

In [12]:
K = 5
kf = KFold(n_splits=K)

In [18]:
feat_cols = ['building_id', 'dt_m', 'dt_w', 'dt_d', 'dt_hour', 
             'dt_day_week', 'dt_day_month', 'dt_week_month']

In [21]:
nrmlses = []
for tr_ix, ts_ix in kf.split(data_4a):
    data_4a1_tr_x = data_4a.iloc[tr_ix[:int(len(tr_ix)*.9)],:][feat_cols]
    data_4a1_tr_y = data_4a.iloc[tr_ix[:int(len(tr_ix)*.9)],:].meter_reading
    data_4a1_val_x = data_4a.iloc[tr_ix[int(len(tr_ix)*.9):],:][feat_cols]
    data_4a1_val_y = data_4a.iloc[tr_ix[int(len(tr_ix)*.9):],:].meter_reading
    lgb_train = lgb.Dataset(data_4a1_tr_x, data_4a1_tr_y)
    lgb_eval = lgb.Dataset(data_4a1_val_x, data_4a1_val_y)
    gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                            early_stopping_rounds=20, verbose_eval = 20)
    data_4a1_ts_x = data_4a.iloc[ts_ix,:][feat_cols]
    data_4a1_ts_y = data_4a.iloc[ts_ix,:].meter_reading
    lgbm_hat = (gbm_regress.predict(data_4a1_ts_x, num_iteration=gbm_regress.best_iteration))
    nrmlse = root_mean_squared_log_error(lgbm_hat, data_4a1_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)
print('FinalScore:', sum(nrmlses) / K)

Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 112.012	valid_1's rmse: 117.662
[40]	training's rmse: 51.171	valid_1's rmse: 71.7402
[60]	training's rmse: 37.0798	valid_1's rmse: 65.697
[80]	training's rmse: 32.7356	valid_1's rmse: 65.1564
[100]	training's rmse: 30.8903	valid_1's rmse: 65.0886
[120]	training's rmse: 29.7747	valid_1's rmse: 64.9115
[140]	training's rmse: 29.0472	valid_1's rmse: 64.8896
[160]	training's rmse: 28.2574	valid_1's rmse: 64.9469
Early stopping, best iteration is:
[130]	training's rmse: 29.3517	valid_1's rmse: 64.8556


/Users/Ric/_personal/x_tsm/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


Iteration done, nrmlse: 0.6900045526028211
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 113.189	valid_1's rmse: 117.728
[40]	training's rmse: 52.7329	valid_1's rmse: 71.0659
[60]	training's rmse: 38.9566	valid_1's rmse: 64.9581
[80]	training's rmse: 34.6426	valid_1's rmse: 64.2604
[100]	training's rmse: 32.7494	valid_1's rmse: 64.2506
[120]	training's rmse: 31.5043	valid_1's rmse: 64.1523
[140]	training's rmse: 30.7502	valid_1's rmse: 64.0148
[160]	training's rmse: 29.8828	valid_1's rmse: 63.7847
[180]	training's rmse: 29.3048	valid_1's rmse: 63.8946
[200]	training's rmse: 28.8511	valid_1's rmse: 63.8845
Early stopping, best iteration is:
[161]	training's rmse: 29.8795	valid_1's rmse: 63.7828
Iteration done, nrmlse: 0.6117565927249083
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 111.312	valid_1's rmse: 117.55
[40]	training's rmse: 49.2208	valid_1's rmse: 71.1952
[60]	training's rmse: 34.8806	valid_1's rmse: 65.

In [34]:
feat_cols = ['building_id_mean', 'dt_m', 'dt_w', 'dt_d', 'dt_hour', 
             'dt_day_week', 'dt_day_month', 'dt_week_month']

In [35]:
# Simple building_id mean
nrmlses = []
for tr_ix, ts_ix in kf.split(data_4a):
    full_training_data = data_4a.iloc[tr_ix[:int(len(tr_ix)*.9)],:]
    building_id_means = full_training_data.groupby('building_id')['meter_reading'].mean().to_dict()
    full_training_data['building_id_mean'] = full_training_data.building_id.apply(lambda x: building_id_means[x])
    data_4a2_tr_x = full_training_data[feat_cols]
    data_4a2_tr_y = data_4a.iloc[tr_ix[:int(len(tr_ix)*.9)],:].meter_reading
    data_4a2_val_x = data_4a.iloc[tr_ix[int(len(tr_ix)*.9):],:]
    data_4a2_val_x['building_id_mean'] = data_4a2_val_x.building_id.apply(lambda x: building_id_means[x])
    data_4a2_val_x = data_4a2_val_x[feat_cols]
    data_4a2_val_y = data_4a.iloc[tr_ix[int(len(tr_ix)*.9):],:].meter_reading
    lgb_train = lgb.Dataset(data_4a2_tr_x, data_4a2_tr_y)
    lgb_eval = lgb.Dataset(data_4a2_val_x, data_4a2_val_y)
    gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                            early_stopping_rounds=20, verbose_eval = 20)
    data_4a2_ts_x = data_4a.iloc[ts_ix,:]
    data_4a2_ts_x['building_id_mean'] = data_4a2_ts_x.building_id.apply(lambda x: building_id_means[x])
    data_4a2_ts_x = data_4a2_ts_x[feat_cols]
    data_4a2_ts_y = data_4a.iloc[ts_ix,:].meter_reading
    lgbm_hat = (gbm_regress.predict(data_4a2_ts_x, num_iteration=gbm_regress.best_iteration))
    nrmlse = root_mean_squared_log_error(lgbm_hat, data_4a2_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)
print('FinalScore:', sum(nrmlses) / K)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 112.337	valid_1's rmse: 118.492
[40]	training's rmse: 51.4967	valid_1's rmse: 72.0433
[60]	training's rmse: 37.3209	valid_1's rmse: 65.7944
[80]	training's rmse: 32.9332	valid_1's rmse: 64.999
[100]	training's rmse: 31.0165	valid_1's rmse: 64.8043
[120]	training's rmse: 29.6872	valid_1's rmse: 64.511
[140]	training's rmse: 28.9264	valid_1's rmse: 64.4965
[160]	training's rmse: 28.0849	valid_1's rmse: 64.5002
[180]	training's rmse: 27.5121	valid_1's rmse: 64.4966
Early stopping, best iteration is:
[157]	training's rmse: 28.1908	valid_1's rmse: 64.4698


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Iteration done, nrmlse: 0.693077635919569
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 113.693	valid_1's rmse: 118.406
[40]	training's rmse: 53.3716	valid_1's rmse: 71.35
[60]	training's rmse: 39.5289	valid_1's rmse: 65.0783
[80]	training's rmse: 35.1127	valid_1's rmse: 64.3017
[100]	training's rmse: 33.1106	valid_1's rmse: 64.1773
[120]	training's rmse: 31.7411	valid_1's rmse: 64.0255
[140]	training's rmse: 30.7709	valid_1's rmse: 64.0641
[160]	training's rmse: 29.8778	valid_1's rmse: 63.9273
[180]	training's rmse: 29.2434	valid_1's rmse: 63.9656
[200]	training's rmse: 28.6719	valid_1's rmse: 63.9739
[220]	training's rmse: 28.3393	valid_1's rmse: 64.0868
Early stopping, best iteration is:
[195]	training's rmse: 28.839	valid_1's rmse: 63.9184
Iteration done, nrmlse: 0.6007772016813125
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 111.658	valid_1's rmse: 118.205
[40]	training's rmse: 49.7047	valid_1's rmse: 71.52

In [110]:
# Building_id mean by month
nrmlses = []
for tr_ix, ts_ix in kf.split(data_4a):
    # Building Means
    full_training_data = data_4a.iloc[tr_ix[:int(len(tr_ix)*.9)],:]
    building_id_means = full_training_data.groupby(['building_id', 'dt_m'], as_index=False)['meter_reading'].mean()
    bid_dicts = {}
    for bid in list(set(building_id_means.building_id)):
        bid_data = building_id_means[building_id_means.building_id == bid][['dt_m', 'meter_reading']].to_dict('records')
        bid_dicts[bid] = bid_data
    # Using Means
    bid_means_tr = pd.DataFrame(full_training_data.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts[x]]).values.tolist(),
                      columns=['building_id_dt_m_{}_mean'.format(m) for m in [mm['dt_m'] for mm in bid_dicts[bid]]]).reset_index(drop=True)
    data_4a2_tr_x = full_training_data.reset_index(drop=True).merge(bid_means_tr, left_index=True, right_index=True)
    data_4a2_tr_x = data_4a2_tr_x[[x for x in list(data_4a2_tr_x) if x not in ['meter_reading', 'building_id']]].fillna(0)
    data_4a2_tr_y = data_4a.iloc[tr_ix[:int(len(tr_ix)*.9)],:].meter_reading
    
    data_4a2_val_x = data_4a.iloc[tr_ix[int(len(tr_ix)*.9):],:]
    bid_means_val = pd.DataFrame(data_4a2_val_x.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts[x]]).values.tolist(),
                  columns=['building_id_dt_m_{}_mean'.format(m) for m in [mm['dt_m'] for mm in bid_dicts[bid]]]).reset_index(drop=True)
    data_4a2_val_x = data_4a2_val_x.reset_index(drop=True).merge(bid_means_val, left_index=True, right_index=True)
    data_4a2_val_x = data_4a2_val_x[[x for x in list(data_4a2_val_x) if x not in ['meter_reading', 'building_id']]].fillna(0)
    data_4a2_val_y = data_4a.iloc[tr_ix[int(len(tr_ix)*.9):],:].meter_reading
    
    lgb_train = lgb.Dataset(data_4a2_tr_x, data_4a2_tr_y)
    lgb_eval = lgb.Dataset(data_4a2_val_x, data_4a2_val_y)
    gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                            early_stopping_rounds=20, verbose_eval = 20)
    
    data_4a2_ts_x = data_4a.iloc[ts_ix,:]
    bid_means_ts = pd.DataFrame(data_4a2_ts_x.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts[x]]).values.tolist(),
                  columns=['building_id_dt_m_{}_mean'.format(m) for m in [mm['dt_m'] for mm in bid_dicts[bid]]]).reset_index(drop=True)
    data_4a2_ts_x = data_4a2_ts_x.reset_index(drop=True).merge(bid_means_ts, left_index=True, right_index=True)
    data_4a2_ts_x = data_4a2_ts_x[[x for x in list(data_4a2_ts_x) if x not in ['meter_reading', 'building_id']]].fillna(0)
    data_4a2_ts_y = data_4a.iloc[ts_ix,:].meter_reading
    
    lgbm_hat = (gbm_regress.predict(data_4a2_ts_x, num_iteration=gbm_regress.best_iteration))
    nrmlse = root_mean_squared_log_error(lgbm_hat, data_4a2_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)
print('FinalScore:', sum(nrmlses) / K)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 68.6821	valid_1's rmse: 80.9615
[40]	training's rmse: 38.3213	valid_1's rmse: 64.131
[60]	training's rmse: 32.5358	valid_1's rmse: 63.6326
[80]	training's rmse: 29.5448	valid_1's rmse: 64.1143
Early stopping, best iteration is:
[49]	training's rmse: 34.787	valid_1's rmse: 63.4503
Iteration done, nrmlse: 0.7158289917549626
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 70.3321	valid_1's rmse: 80.4299
[40]	training's rmse: 40.9483	valid_1's rmse: 63.8159
[60]	training's rmse: 34.7831	valid_1's rmse: 63.0026
[80]	training's rmse: 31.7889	valid_1's rmse: 63.6859
Early stopping, best iteration is:
[55]	training's rmse: 35.9387	valid_1's rmse: 62.9315


/Users/Ric/_personal/x_tsm/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


Iteration done, nrmlse: 0.5852495433240882
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 66.9308	valid_1's rmse: 80.8209
[40]	training's rmse: 35.7994	valid_1's rmse: 63.383
[60]	training's rmse: 30.8471	valid_1's rmse: 62.7939
[80]	training's rmse: 28.5576	valid_1's rmse: 63.201
Early stopping, best iteration is:
[54]	training's rmse: 31.7197	valid_1's rmse: 62.6625
Iteration done, nrmlse: 0.9855014809296719
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 67.4123	valid_1's rmse: 80.7894
[40]	training's rmse: 37.233	valid_1's rmse: 63.6767
[60]	training's rmse: 31.8162	valid_1's rmse: 62.989
[80]	training's rmse: 29.307	valid_1's rmse: 63.6074
[100]	training's rmse: 27.6085	valid_1's rmse: 64.1073
Early stopping, best iteration is:
[63]	training's rmse: 31.4281	valid_1's rmse: 62.9437
Iteration done, nrmlse: 1.5148254891166526
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 68.145

In [111]:
# Building_id mean by week of the year
nrmlses = []
for tr_ix, ts_ix in kf.split(data_4a):
    # Building Means
    full_training_data = data_4a.iloc[tr_ix[:int(len(tr_ix)*.9)],:]
    building_id_means = full_training_data.groupby(['building_id', 'dt_w'], as_index=False)['meter_reading'].mean()
    bid_dicts = {}
    for bid in list(set(building_id_means.building_id)):
        bid_data = building_id_means[building_id_means.building_id == bid][['dt_w', 'meter_reading']].to_dict('records')
        bid_dicts[bid] = bid_data
    # Using Means
    bid_means_tr = pd.DataFrame(full_training_data.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts[x]]).values.tolist(),
                      columns=['building_id_dt_w_{}_mean'.format(m) for m in [mm['dt_w'] for mm in bid_dicts[bid]]]).reset_index(drop=True)
    data_4a2_tr_x = full_training_data.reset_index(drop=True).merge(bid_means_tr, left_index=True, right_index=True)
    data_4a2_tr_x = data_4a2_tr_x[[x for x in list(data_4a2_tr_x) if x not in ['meter_reading', 'building_id']]].fillna(0)
    data_4a2_tr_y = data_4a.iloc[tr_ix[:int(len(tr_ix)*.9)],:].meter_reading
    
    data_4a2_val_x = data_4a.iloc[tr_ix[int(len(tr_ix)*.9):],:]
    bid_means_val = pd.DataFrame(data_4a2_val_x.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts[x]]).values.tolist(),
                  columns=['building_id_dt_w_{}_mean'.format(m) for m in [mm['dt_w'] for mm in bid_dicts[bid]]]).reset_index(drop=True)
    data_4a2_val_x = data_4a2_val_x.reset_index(drop=True).merge(bid_means_val, left_index=True, right_index=True)
    data_4a2_val_x = data_4a2_val_x[[x for x in list(data_4a2_val_x) if x not in ['meter_reading', 'building_id']]].fillna(0)
    data_4a2_val_y = data_4a.iloc[tr_ix[int(len(tr_ix)*.9):],:].meter_reading
    
    lgb_train = lgb.Dataset(data_4a2_tr_x, data_4a2_tr_y)
    lgb_eval = lgb.Dataset(data_4a2_val_x, data_4a2_val_y)
    gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                            early_stopping_rounds=20, verbose_eval = 20)
    
    data_4a2_ts_x = data_4a.iloc[ts_ix,:]
    bid_means_ts = pd.DataFrame(data_4a2_ts_x.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts[x]]).values.tolist(),
                  columns=['building_id_dt_w_{}_mean'.format(m) for m in [mm['dt_w'] for mm in bid_dicts[bid]]]).reset_index(drop=True)
    data_4a2_ts_x = data_4a2_ts_x.reset_index(drop=True).merge(bid_means_ts, left_index=True, right_index=True)
    data_4a2_ts_x = data_4a2_ts_x[[x for x in list(data_4a2_ts_x) if x not in ['meter_reading', 'building_id']]].fillna(0)
    data_4a2_ts_y = data_4a.iloc[ts_ix,:].meter_reading
    
    lgbm_hat = (gbm_regress.predict(data_4a2_ts_x, num_iteration=gbm_regress.best_iteration))
    nrmlse = root_mean_squared_log_error(lgbm_hat, data_4a2_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)
print('FinalScore:', sum(nrmlses) / K)

Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 69.2653	valid_1's rmse: 80.8902
[40]	training's rmse: 38.7019	valid_1's rmse: 64.2818
[60]	training's rmse: 32.5261	valid_1's rmse: 64.0305
[80]	training's rmse: 29.3015	valid_1's rmse: 64.6076
Early stopping, best iteration is:
[52]	training's rmse: 34.2409	valid_1's rmse: 63.7832
Iteration done, nrmlse: 0.714107501741141
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 70.4381	valid_1's rmse: 81.3825
[40]	training's rmse: 40.782	valid_1's rmse: 64.1236
[60]	training's rmse: 34.4132	valid_1's rmse: 63.4819
[80]	training's rmse: 31.2167	valid_1's rmse: 63.7218
Early stopping, best iteration is:
[58]	training's rmse: 34.6982	valid_1's rmse: 63.3677
Iteration done, nrmlse: 0.5810043348982631
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 67.1101	valid_1's rmse: 81.468
[40]	training's rmse: 35.8849	valid_1's rmse: 64.0749
[60]	training's 

In [120]:
# Building_id mean by month
nrmlses = []
for tr_ix, ts_ix in kf.split(data_4a):
    # Building Means
    full_training_data = data_4a.iloc[tr_ix[:int(len(tr_ix)*.9)],:]
    building_id_means = full_training_data.groupby(['building_id', 'dt_m'], as_index=False).agg({'meter_reading': 'mean'})
    bid_dicts = {}
    for bid in list(set(building_id_means.building_id)):
        bid_data = building_id_means[building_id_means.building_id == bid][['dt_m', 'meter_reading']].to_dict('records')
        bid_dicts[bid] = bid_data
    building_id_stds = full_training_data.groupby(['building_id', 'dt_m'], as_index=False).agg({'meter_reading': 'std'})
    bid_dicts_s = {}
    for bids in list(set(building_id_stds.building_id)):
        bid_datas = building_id_stds[building_id_stds.building_id == bids][['dt_m', 'meter_reading']].to_dict('records')
        bid_dicts_s[bids] = bid_datas
    # Using Means / STDS
    bid_means_tr = pd.DataFrame(full_training_data.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts[x]]).values.tolist(),
                      columns=['building_id_dt_m_{}_mean'.format(m) for m in [mm['dt_m'] for mm in bid_dicts[bid]]]).reset_index(drop=True)
    data_4a2_tr_x = full_training_data.reset_index(drop=True).merge(bid_means_tr, left_index=True, right_index=True)
    bid_std_tr = pd.DataFrame(full_training_data.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts_s[x]]).values.tolist(),
                      columns=['building_id_dt_m_{}_std'.format(m) for m in [mm['dt_m'] for mm in bid_dicts_s[bids]]]).reset_index(drop=True)
    data_4a2_tr_x = data_4a2_tr_x.reset_index(drop=True).merge(bid_std_tr, left_index=True, right_index=True)
    data_4a2_tr_x = data_4a2_tr_x[[x for x in list(data_4a2_tr_x) if x not in ['meter_reading', 'building_id']]].fillna(0)
    data_4a2_tr_y = data_4a.iloc[tr_ix[:int(len(tr_ix)*.9)],:].meter_reading
    
    data_4a2_val_x = data_4a.iloc[tr_ix[int(len(tr_ix)*.9):],:]
    bid_means_val = pd.DataFrame(data_4a2_val_x.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts[x]]).values.tolist(),
                  columns=['building_id_dt_m_{}_mean'.format(m) for m in [mm['dt_m'] for mm in bid_dicts[bid]]]).reset_index(drop=True)
    data_4a2_val_x = data_4a2_val_x.reset_index(drop=True).merge(bid_means_val, left_index=True, right_index=True)
    bid_std_val = pd.DataFrame(data_4a2_val_x.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts_s[x]]).values.tolist(),
                      columns=['building_id_dt_m_{}_std'.format(m) for m in [mm['dt_m'] for mm in bid_dicts_s[bids]]]).reset_index(drop=True)
    data_4a2_val_x = data_4a2_val_x.reset_index(drop=True).merge(bid_std_val, left_index=True, right_index=True)
    data_4a2_val_x = data_4a2_val_x[[x for x in list(data_4a2_val_x) if x not in ['meter_reading', 'building_id']]].fillna(0)
    data_4a2_val_y = data_4a.iloc[tr_ix[int(len(tr_ix)*.9):],:].meter_reading
    
    lgb_train = lgb.Dataset(data_4a2_tr_x, data_4a2_tr_y)
    lgb_eval = lgb.Dataset(data_4a2_val_x, data_4a2_val_y)
    gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                            early_stopping_rounds=20, verbose_eval = 20)
    
    data_4a2_ts_x = data_4a.iloc[ts_ix,:]
    bid_means_ts = pd.DataFrame(data_4a2_ts_x.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts[x]]).values.tolist(),
                  columns=['building_id_dt_m_{}_mean'.format(m) for m in [mm['dt_m'] for mm in bid_dicts[bid]]]).reset_index(drop=True)
    data_4a2_ts_x = data_4a2_ts_x.reset_index(drop=True).merge(bid_means_ts, left_index=True, right_index=True)
    bid_std_val = pd.DataFrame(data_4a2_ts_x.building_id.apply(lambda x: [x['meter_reading'] for x in bid_dicts_s[x]]).values.tolist(),
                      columns=['building_id_dt_m_{}_std'.format(m) for m in [mm['dt_m'] for mm in bid_dicts_s[bids]]]).reset_index(drop=True)
    data_4a2_val_x = data_4a2_ts_x.reset_index(drop=True).merge(bid_std_val, left_index=True, right_index=True)
    data_4a2_ts_x = data_4a2_ts_x[[x for x in list(data_4a2_ts_x) if x not in ['meter_reading', 'building_id']]].fillna(0)
    data_4a2_ts_y = data_4a.iloc[ts_ix,:].meter_reading
    
    lgbm_hat = (gbm_regress.predict(data_4a2_ts_x, num_iteration=gbm_regress.best_iteration))
    nrmlse = root_mean_squared_log_error(lgbm_hat, data_4a2_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)
print('FinalScore:', sum(nrmlses) / K)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 69.0254	valid_1's rmse: 80.4186
[40]	training's rmse: 38.622	valid_1's rmse: 63.77
[60]	training's rmse: 32.5158	valid_1's rmse: 63.244
[80]	training's rmse: 29.4813	valid_1's rmse: 63.95
Early stopping, best iteration is:
[52]	training's rmse: 34.2986	valid_1's rmse: 63.1415
Iteration done, nrmlse: 0.7169357400494929
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 70.4836	valid_1's rmse: 80.7937
[40]	training's rmse: 40.6255	valid_1's rmse: 63.7733
[60]	training's rmse: 34.4261	valid_1's rmse: 63.3333
[80]	training's rmse: 31.2054	valid_1's rmse: 63.52
[100]	training's rmse: 29.0336	valid_1's rmse: 64.2659
Early stopping, best iteration is:
[65]	training's rmse: 33.4895	valid_1's rmse: 63.1078


/Users/Ric/_personal/x_tsm/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


Iteration done, nrmlse: 0.5915644525215957
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 67.0306	valid_1's rmse: 80.4954
[40]	training's rmse: 36.0095	valid_1's rmse: 63.2581
[60]	training's rmse: 30.6872	valid_1's rmse: 63.0507
[80]	training's rmse: 28.3548	valid_1's rmse: 63.5272
Early stopping, best iteration is:
[47]	training's rmse: 33.3043	valid_1's rmse: 62.6665
Iteration done, nrmlse: 1.011173306404626
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 67.6534	valid_1's rmse: 81.258
[40]	training's rmse: 37.1633	valid_1's rmse: 64.183
[60]	training's rmse: 31.6811	valid_1's rmse: 63.5877
[80]	training's rmse: 28.9484	valid_1's rmse: 64.1188
Early stopping, best iteration is:
[51]	training's rmse: 33.3169	valid_1's rmse: 63.2353
Iteration done, nrmlse: 1.5775627920723108
Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 68.1208	valid_1's rmse: 116.587
[40]	training's rmse: 37.91